In [2]:
%pip install -q opentelemetry-api
%pip install -q opentelemetry-sdk 
%pip install -q opentelemetry-exporter-otlp-proto-grpc


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# save as reasoning_flow_trace.py
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
import time

# Setup
resource = Resource(attributes={"service.name": "reasoning-agent"})
provider = TracerProvider(resource=resource)
exporter = OTLPSpanExporter(endpoint="localhost:4317", insecure=True)
processor = BatchSpanProcessor(exporter)
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
tracer = trace.get_tracer("reasoning-tracer")

# Main agent span
with tracer.start_as_current_span(
    "chain_of_thought",
    attributes={
        "gen_ai.operation.name": "agent",
        "gen_ai.system": "openai",
        "gen_ai.agent.name": "reasoning-agent",
        "gen_ai.request.model": "gpt-4o"
    }
):
    # Initial problem analysis
    with tracer.start_as_current_span(
        "step1_analyze",
        attributes={"gen_ai.operation.name": "thinking"}
    ):
        current_span = trace.get_current_span()
        current_span.add_event(
            "reasoning_step", 
            attributes={"thought": "Let me analyze this math problem step by step."}
        )
        time.sleep(0.1)  # Simulate processing time
    
    # Generate potential solutions
    with tracer.start_as_current_span(
        "step2_generate_options",
        attributes={"gen_ai.operation.name": "thinking"}
    ):
        current_span = trace.get_current_span()
        current_span.add_event(
            "reasoning_step", 
            attributes={"thought": "I need to find the derivative of x²sin(x)"}
        )
        time.sleep(0.1)  # Simulate processing time
    
    # Evaluate options
    with tracer.start_as_current_span(
        "step3_evaluate",
        attributes={"gen_ai.operation.name": "thinking"}
    ):
        current_span = trace.get_current_span()
        current_span.add_event(
            "reasoning_step", 
            attributes={"thought": "Using the product rule: d/dx[x²sin(x)] = 2xsin(x) + x²cos(x)"}
        )
        time.sleep(0.1)  # Simulate processing time
    
    # Final decision
    with tracer.start_as_current_span(
        "step4_decide",
        attributes={"gen_ai.operation.name": "thinking"}
    ):
        current_span = trace.get_current_span()
        current_span.add_event(
            "reasoning_step", 
            attributes={"thought": "The final answer is 2xsin(x) + x²cos(x)"}
        )
        time.sleep(0.1)  # Simulate processing time

print("Multi-step reasoning trace generated. Check Jaeger UI.")

Multi-step reasoning trace generated. Check Jaeger UI.


In [4]:
# save as tool_usage_trace.py
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
import time
import json

# Setup
resource = Resource(attributes={"service.name": "agent-with-tools"})
provider = TracerProvider(resource=resource)
exporter = OTLPSpanExporter(endpoint="localhost:4317", insecure=True)
processor = BatchSpanProcessor(exporter)
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
tracer = trace.get_tracer("tool-tracer")

# Main agent operation
with tracer.start_as_current_span(
    "chat gpt-4o",
    attributes={
        "gen_ai.system": "openai",
        "gen_ai.operation.name": "chat",
        "gen_ai.request.model": "gpt-4o"
    }
):
    # Add event for the user message
    current_span = trace.get_current_span()
    current_span.add_event(
        "gen_ai.user.message", 
        attributes={"content": "What's the weather in Paris?"}
    )
    
    # Add event for the assistant deciding to use a tool
    current_span.add_event(
        "gen_ai.assistant.message", 
        attributes={
            # Either omit the content field entirely or use an empty string
            # "content": "",  # Empty content instead of None
            # Alternatively, simply omit the content field:
            "tool_calls": json.dumps([{
                "id": "call_abc123",
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "arguments": '{"location":"Paris"}'
                }
            }])
        }
    )
    
    # Add a child span for the tool execution
    with tracer.start_as_current_span(
        "execute_tool get_weather",
        attributes={
            "gen_ai.operation.name": "execute_tool",
            "gen_ai.tool.name": "get_weather",
            "gen_ai.tool.call.id": "call_abc123"
        }
    ):
        # Simulate tool execution
        time.sleep(0.2)
        
        # Add tool response event
        tool_span = trace.get_current_span()
        tool_span.add_event(
            "gen_ai.tool.message", 
            attributes={
                "content": "rainy, 57°F",
                "id": "call_abc123",
                "role": "tool"
            }
        )
    
    # Add event for the final assistant response
    current_span.add_event(
        "gen_ai.assistant.message", 
        attributes={
            "content": "The weather in Paris is rainy with a temperature of 57°F."
        }
    )

print("Tool usage trace generated. Check Jaeger UI.")

Overriding of current TracerProvider is not allowed


Tool usage trace generated. Check Jaeger UI.
